## Data loading

In [1]:
import pandas as pd
import datetime

In [2]:
path_data_1 = 'data_flight/flights.csv'
path_data_2 = 'data_flight/airports.csv'
path_data_3 = 'data_flight/airlines.csv'


#df_flights =  pd.read_csv(path_data_1, delimiter=',', nrows=2000)
df_flights =  pd.read_csv(path_data_1, delimiter=',')
df_airports =  pd.read_csv(path_data_2, delimiter=',')
df_airlines =  pd.read_csv(path_data_3, delimiter=',')

## Data exploration

In [ ]:
df_flights.info()

In [72]:
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In [73]:
df_airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes


In [74]:
df_flights.head()
#df_airports.head()
#df_airlines.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Join tables based on mutual data

In [76]:
df_joined = df_flights.join(df_airports.set_index('IATA_CODE').add_prefix('airport_org_'),
                            on='ORIGIN_AIRPORT',
                            rsuffix='_org_')

In [77]:
df_joined = df_joined.join(df_airports.set_index('IATA_CODE').add_prefix('airport_des_'),
                            on='DESTINATION_AIRPORT',
                            rsuffix='_des_')

In [78]:
df_joined = df_joined.join(df_airlines.set_index('IATA_CODE').add_suffix('_name'),
                            on='AIRLINE')

In [79]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 44 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   YEAR                   int64  
 1   MONTH                  int64  
 2   DAY                    int64  
 3   DAY_OF_WEEK            int64  
 4   AIRLINE                object 
 5   FLIGHT_NUMBER          int64  
 6   TAIL_NUMBER            object 
 7   ORIGIN_AIRPORT         object 
 8   DESTINATION_AIRPORT    object 
 9   SCHEDULED_DEPARTURE    int64  
 10  DEPARTURE_TIME         float64
 11  DEPARTURE_DELAY        float64
 12  TAXI_OUT               float64
 13  WHEELS_OFF             float64
 14  SCHEDULED_TIME         float64
 15  ELAPSED_TIME           float64
 16  AIR_TIME               float64
 17  DISTANCE               int64  
 18  WHEELS_ON              float64
 19  TAXI_IN                float64
 20  SCHEDULED_ARRIVAL      int64  
 21  ARRIVAL_TIME           float64
 22  ARRIVAL_DELAY     

## Data manipulation

In [89]:
df_joined['DEPARTURE_TIME'] = pd.to_datetime(df_joined['DEPARTURE_TIME'], format='%H%M', errors='coerce').dt.time
df_joined['SCHEDULED_TIME'] = pd.to_datetime(df_joined['SCHEDULED_TIME'], format='%H%M', errors='coerce').dt.time
df_joined['SCHEDULED_DEPARTURE'] = pd.to_datetime(df_joined['SCHEDULED_DEPARTURE'], format='%H%M', errors='coerce').dt.time

In [80]:
df_joined.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,airport_org_COUNTRY,airport_org_LATITUDE,airport_org_LONGITUDE,airport_des_AIRPORT,airport_des_CITY,airport_des_STATE,airport_des_COUNTRY,airport_des_LATITUDE,airport_des_LONGITUDE,AIRLINE_name
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,USA,61.17432,-149.99619,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,Alaska Airlines Inc.
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,USA,33.94254,-118.40807,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559,American Airlines Inc.
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,USA,37.61900,-122.37484,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313,US Airways Inc.
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,USA,33.94254,-118.40807,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,American Airlines Inc.
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,USA,47.44898,-122.30931,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,Alaska Airlines Inc.


In [90]:
df_joined['Date'] = pd.to_datetime(df_joined[['YEAR', 'MONTH', 'DAY']])

In [91]:
print(df_joined.columns)

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'airport_org_AIRPORT',
       'airport_org_CITY', 'airport_org_STATE', 'airport_org_COUNTRY',
       'airport_org_LATITUDE', 'airport_org_LONGITUDE', 'airport_des_AIRPORT',
       'airport_des_CITY', 'airport_des_STATE', 'airport_des_COUNTRY',
       'airport_des_LATITUDE', 'airport_des_LONGITUDE', 'AIRLINE_name',
       'Date'],
      dtype='object')


In [92]:
df = df_joined.drop(columns=['YEAR', 'MONTH', 'DAY','AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','WHEELS_OFF', 'WHEELS_ON','TAXI_IN', 'TAXI_OUT', 'airport_org_STATE','airport_des_STATE', \
    'airport_org_LATITUDE', 'airport_des_LATITUDE','airport_org_LONGITUDE','airport_des_LONGITUDE', 'ELAPSED_TIME'])

In [93]:
df.set_index(['TAIL_NUMBER', 'SCHEDULED_DEPARTURE'], inplace=True)

In [94]:
print(df.columns)

Index(['DAY_OF_WEEK', 'FLIGHT_NUMBER', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
       'SCHEDULED_TIME', 'AIR_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL',
       'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED',
       'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
       'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY',
       'airport_org_AIRPORT', 'airport_org_CITY', 'airport_org_COUNTRY',
       'airport_des_AIRPORT', 'airport_des_CITY', 'airport_des_COUNTRY',
       'AIRLINE_name', 'Date'],
      dtype='object')


In [95]:
df.head()

,,DAY_OF_WEEK,FLIGHT_NUMBER,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,airport_org_AIRPORT,airport_org_CITY,airport_org_COUNTRY,airport_des_AIRPORT,airport_des_CITY,airport_des_COUNTRY,AIRLINE_name,Date
TAIL_NUMBER,SCHEDULED_DEPARTURE,,,,,,,,,,,,,,,,,,,,,
N407AS,NaN,4,98,23:54:00,-11.0,20:05:00,169.0,1448,430,408.0,-22.0,...,NaN,NaN,Ted Stevens Anchorage International Airport,Anchorage,USA,Seattle-Tacoma International Airport,Seattle,USA,Alaska Airlines Inc.,2015-01-01
N3KUAA,01:00:00,4,2336,NaT,-8.0,NaT,263.0,2330,750,741.0,-9.0,...,NaN,NaN,Los Angeles International Airport,Los Angeles,USA,Palm Beach International Airport,West Palm Beach,USA,American Airlines Inc.,2015-01-01
N171US,02:00:00,4,840,01:08:00,-2.0,NaT,266.0,2296,806,811.0,5.0,...,NaN,NaN,San Francisco International Airport,San Francisco,USA,Charlotte Douglas International Airport,Charlotte,USA,US Airways Inc.,2015-01-01
N3HYAA,02:00:00,4,258,01:05:00,-5.0,NaT,258.0,2342,805,756.0,-9.0,...,NaN,NaN,Los Angeles International Airport,Los Angeles,USA,Miami International Airport,Miami,USA,American Airlines Inc.,2015-01-01
N527AS,02:05:00,4,135,02:04:00,-1.0,23:05:00,199.0,1448,320,259.0,-21.0,...,NaN,NaN,Seattle-Tacoma International Airport,Seattle,USA,Ted Stevens Anchorage International Airport,Anchorage,USA,Alaska Airlines Inc.,2015-01-01


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5819079 entries, ('N407AS', nan) to ('N534JB', datetime.time(23, 59))
Data columns (total 26 columns):
 #   Column               Dtype         
---  ------               -----         
 0   DAY_OF_WEEK          int64         
 1   FLIGHT_NUMBER        int64         
 2   DEPARTURE_TIME       object        
 3   DEPARTURE_DELAY      float64       
 4   SCHEDULED_TIME       object        
 5   AIR_TIME             float64       
 6   DISTANCE             int64         
 7   SCHEDULED_ARRIVAL    int64         
 8   ARRIVAL_TIME         float64       
 9   ARRIVAL_DELAY        float64       
 10  DIVERTED             int64         
 11  CANCELLED            int64         
 12  CANCELLATION_REASON  object        
 13  AIR_SYSTEM_DELAY     float64       
 14  SECURITY_DELAY       float64       
 15  AIRLINE_DELAY        float64       
 16  LATE_AIRCRAFT_DELAY  float64       
 17  WEATHER_DELAY        float64       
 18  airport_org_AIRPORT  o

## Save the preparated table 

In [97]:
df.to_csv('flights_filtered.csv')

In [99]:
df.head(20)

,,DAY_OF_WEEK,FLIGHT_NUMBER,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,airport_org_AIRPORT,airport_org_CITY,airport_org_COUNTRY,airport_des_AIRPORT,airport_des_CITY,airport_des_COUNTRY,AIRLINE_name,Date
TAIL_NUMBER,SCHEDULED_DEPARTURE,,,,,,,,,,,,,,,,,,,,,
N407AS,NaN,4,98,23:54:00,-11.0,20:05:00,169.0,1448,430,408.0,-22.0,...,NaN,NaN,Ted Stevens Anchorage International Airport,Anchorage,USA,Seattle-Tacoma International Airport,Seattle,USA,Alaska Airlines Inc.,2015-01-01
N3KUAA,01:00:00,4,2336,NaT,-8.0,NaT,263.0,2330,750,741.0,-9.0,...,NaN,NaN,Los Angeles International Airport,Los Angeles,USA,Palm Beach International Airport,West Palm Beach,USA,American Airlines Inc.,2015-01-01
N171US,02:00:00,4,840,01:08:00,-2.0,NaT,266.0,2296,806,811.0,5.0,...,NaN,NaN,San Francisco International Airport,San Francisco,USA,Charlotte Douglas International Airport,Charlotte,USA,US Airways Inc.,2015-01-01
N3HYAA,02:00:00,4,258,01:05:00,-5.0,NaT,258.0,2342,805,756.0,-9.0,...,NaN,NaN,Los Angeles International Airport,Los Angeles,USA,Miami International Airport,Miami,USA,American Airlines Inc.,2015-01-01
N527AS,02:05:00,4,135,02:04:00,-1.0,23:05:00,199.0,1448,320,259.0,-21.0,...,NaN,NaN,Seattle-Tacoma International Airport,Seattle,USA,Ted Stevens Anchorage International Airport,Anchorage,USA,Alaska Airlines Inc.,2015-01-01
N3730B,02:05:00,4,806,02:00:00,-5.0,21:07:00,206.0,1589,602,610.0,8.0,...,NaN,NaN,San Francisco International Airport,San Francisco,USA,Minneapolis-Saint Paul International Airport,Minneapolis,USA,Delta Air Lines Inc.,2015-01-01
N635NK,02:05:00,4,612,01:09:00,-6.0,18:01:00,154.0,1299,526,509.0,-17.0,...,NaN,NaN,McCarran International Airport,Las Vegas,USA,Minneapolis-Saint Paul International Airport,Minneapolis,USA,Spirit Air Lines,2015-01-01
N584UW,03:00:00,4,2013,04:04:00,14.0,NaT,228.0,2125,803,753.0,-10.0,...,NaN,NaN,Los Angeles International Airport,Los Angeles,USA,Charlotte Douglas International Airport,Charlotte,USA,US Airways Inc.,2015-01-01
N3LAAA,03:00:00,4,1112,01:09:00,-11.0,19:05:00,173.0,1464,545,532.0,-13.0,...,NaN,NaN,San Francisco International Airport,San Francisco,USA,Dallas/Fort Worth International Airport,Dallas-Fort Worth,USA,American Airlines Inc.,2015-01-01
